<a href="https://colab.research.google.com/github/virendrasinh734/skin_disease_detection_using_cnns/blob/main/raw_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from PIL import Image
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Flatten, Dense

# Load the labels CSV
labels_df = pd.read_csv('result.csv')
labels_df=labels_df[:1000]
class_labels = labels_df.columns[1:]
image_filenames = labels_df['image']
for i in range(len(image_filenames)):
    image_filenames[i] = image_filenames[i] + ".jpg"

labels = labels_df[class_labels].values
X_train, X_val, y_train, y_val = train_test_split(image_filenames, labels, test_size=0.2, random_state=42)

# Define a custom data generator for preprocessing
def custom_data_generator(image_paths, labels, batch_size, target_size=(224, 224)):
    num_samples = len(image_paths)
    while True:
        for i in range(0, num_samples, batch_size):
            batch_image_paths = image_paths[i:i + batch_size]
            batch_labels = labels[i:i + batch_size]
            batch_images = [preprocess_image(image_path, target_size) for image_path in batch_image_paths]
            yield np.array(batch_images), batch_labels

def preprocess_image(image_path, target_size=(224, 224)):
    img = Image.open('./ISIC_2019_Training_Input./' + image_path)
    img = img.resize(target_size)
    img_array = np.array(img) / 255.0
    return img_array

batch_size = 16
train_data_generator = custom_data_generator(X_train, y_train, batch_size)
val_data_generator = custom_data_generator(X_val, y_val, batch_size)

# base_model = tf.keras.applications.vgg16.VGG16(weights='imagenet')
model = Sequential()
model.add(Conv2D(256, (3, 3), activation="relu", input_shape=(224, 224, 3)))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

model.add(Conv2D(128, (3, 3),activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

model.add(Conv2D(64, (3, 3),activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.3))
model.add(Flatten())

# model.add(base_model)
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(9,activation='softmax'))
base_model.trainable=False
model.compile(loss='categorical_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])


epochs = 4
steps_per_epoch = len(X_train) // batch_size
validation_steps = len(X_val) // batch_size

history = model.fit(train_data_generator,
                    epochs=epochs,
                    steps_per_epoch=steps_per_epoch,
                    validation_data=val_data_generator,
                    validation_steps=validation_steps)


